In [ ]:
!pip install langchain pinecone-client sentence-transformers
!pip install langchain-pinecone
!pip install -U langchain-community

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = ""    #your_api_key

Repeat this for all organs (kidney, eye, brain, chest, liver)

In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore(
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="abhinand/MedEmbed-large-v0.1")

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("kidney")      #repeat this for all organs kidney, eye, brain, chest, liver)
index_name = "kidney"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embedding_model)

In [ ]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv("/content/augmented_kidney_data.csv")          #put your own file path

# Replace NaN with an empty string in the relevant columns
df.fillna("", inplace=True)

# Prepare data for embedding
for _, row in df.iterrows():
    metadata = {
        "treatment_recommendations": str(row["Treatment Recommendations"]) or "No data",
        "possible_causes": str(row["Possible Causes"]) or "No data",
        "prescriptions": str(row["Prescriptions"]) or "No data",
        "blood_tests": str(row["Blood Tests"]) or "No data",
    }
    text = f"Age Range: {row['Age Range']}, Clinical History: {row['clinical_history_hybrid']}, Disease: {row['disease_hybrid']}, Features: {row['Features']}"

    # Validate text and metadata before uploading
    if all(isinstance(value, str) and value.strip() != "" for value in metadata.values()):
        # Upsert embedding with metadata
        vectorstore.add_texts([text], metadatas=[metadata], ids=[str(row.name)])
    else:
        print(f"Skipping invalid row: {row.name}")

print("Data upload complete!")


In [ ]:
# Example queryry: Type 2 Diabetes, Hypertension, Disease: Non-Alcoholic Fatty Liver Disease"

# Perform similarity search
results = vectorstore.similarity_search(query, k=5)

# Display results
for result in results:
    print(f"Text: {result.page_content}")
    print(f"Metadata: {result.metadata}")
